# Budget Analysis

---

## Environment Setup

In [4]:
# Initial Imports
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv

%matplotlib inline

In [5]:
# Helper functions

def pretty_print_response(response):
    """
    This function formats a JSON file to pretty printing it in the screen.
    """
    print(json.dumps(response, indent=4, sort_keys=True))

In [6]:
# Set environment variables from the .env file
load_dotenv()

True

In [7]:
# Extract API keys from environment variables
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET")
PLAID_PUBLIC_KEY = os.getenv("PLAID_PUBLIC_KEY")

---

## Plaid Access Token

In this section, you will use the `plaid-python` API to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:

1. Create a client to connect to plaid

2. Use the client to generate a public token and request the following items: ['transactions', 'income', 'assets']

3. Exchange the public token for an access token

4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [8]:
# Create the client object
client = plaid.Client(
    client_id=PLAID_CLIENT_ID,
    secret=PLAID_SECRET,
    public_key=PLAID_PUBLIC_KEY,
    environment="sandbox"
)

In [9]:
# Set the institution id
INSITUTION_ID = "ins_109508"

### 2. Generate a public token

In [10]:
# Create the public token
create_tkn_response = client.Sandbox.public_token.create(
    INSITUTION_ID,
    ["transactions","income","assets"]
)
# Print the public token
print(create_tkn_response)

{'public_token': 'public-sandbox-aa0ff33d-7484-4351-b2ca-41df60558dd1', 'request_id': 'UVRlUqVbTlvOOov'}


### 3. Exchange the public token for an access token

In [11]:
# Retrieve the access token
exchange_response = client.Item.public_token.exchange(create_tkn_response["public_token"])
# Print the access toke
print(exchange_response)

{'access_token': 'access-sandbox-adc28ad5-7bc6-41bd-9aec-4865bc5a3b1b', 'item_id': 'kZ9pXme9BksmEmD1pPWRI4pv137wo4iW6edbE', 'request_id': 'rlPaHFyzRwvLTXv'}


In [12]:
# Store the access token in a Python variable
access_token = exchange_response["access_token"]

### 4. Fetch Accounts

In [13]:
# Retrieve accounts
Accounts_response = client.Accounts.get(access_token)
# Print accounts
print(json.dumps(Accounts_response["accounts"][:3],indent=4, sort_keys=True))

[
    {
        "account_id": "d4WNeJlWb8h6o6NWeZX8CPdy6mqr5aiZQndoM",
        "balances": {
            "available": 100,
            "current": 110,
            "iso_currency_code": "USD",
            "limit": null,
            "unofficial_currency_code": null
        },
        "mask": "0000",
        "name": "Plaid Checking",
        "official_name": "Plaid Gold Standard 0% Interest Checking",
        "subtype": "checking",
        "type": "depository"
    },
    {
        "account_id": "axgPJ3EgD9INANPv46reCWK8vrPljbH7QKbp4",
        "balances": {
            "available": 200,
            "current": 210,
            "iso_currency_code": "USD",
            "limit": null,
            "unofficial_currency_code": null
        },
        "mask": "1111",
        "name": "Plaid Saving",
        "official_name": "Plaid Silver Standard 0.1% Interest Saving",
        "subtype": "savings",
        "type": "depository"
    },
    {
        "account_id": "4q7pwV37mGHWpWeE9D3xSzp78xKkdnFdARq8z"

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days

2. Print the categories for each transaction type

3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)

4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [14]:
# Set the start and end date 90 days from today
start_date = "{:%Y-%m-%d}".format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = "{:%Y-%m-%d}".format(datetime.datetime.now())
# Retrieve the transactions for the last 90 days
transaction_response = client.Transactions.get(access_token,start_date,end_date)
# Print the transactions
print(json.dumps(transaction_response["transactions"][:3],indent=4, sort_keys=True))

[
    {
        "account_id": "NkzMVnezXguBWBG56y1KU4BDLznoP6uW5AbLw",
        "account_owner": null,
        "amount": 500,
        "authorized_date": null,
        "category": [
            "Travel",
            "Airlines and Aviation Services"
        ],
        "category_id": "22001000",
        "date": "2020-06-29",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "merchant_name": "United Airlines",
        "name": "United Airlines",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "refer

### 2. Print the categories for each transaction

In [15]:
# Create a for-loop to print the categories for each transaction
# print(transaction_response.keys())
transaction_categories = [transaction["category"] for transaction in transaction_response['transactions']]

pretty_print_response(transaction_categories)

[
    [
        "Travel",
        "Airlines and Aviation Services"
    ],
    [
        "Travel",
        "Taxi"
    ],
    [
        "Food and Drink",
        "Restaurants"
    ],
    [
        "Payment"
    ],
    [
        "Food and Drink",
        "Restaurants",
        "Fast Food"
    ],
    [
        "Shops",
        "Sporting Goods"
    ],
    [
        "Payment",
        "Credit Card"
    ],
    [
        "Travel",
        "Taxi"
    ],
    [
        "Transfer",
        "Debit"
    ],
    [
        "Transfer",
        "Deposit"
    ],
    [
        "Recreation",
        "Gyms and Fitness Centers"
    ],
    [
        "Travel",
        "Airlines and Aviation Services"
    ],
    [
        "Food and Drink",
        "Restaurants",
        "Fast Food"
    ],
    [
        "Food and Drink",
        "Restaurants",
        "Coffee Shop"
    ],
    [
        "Food and Drink",
        "Restaurants"
    ],
    [
        "Transfer",
        "Credit"
    ],
    [
        "Travel",
        

### 3. Create a new DataFrame using the following fields from the JSON transaction data: `date`, `name`, `amount`, `category`. 

(For categories with more than one label, just use the first category label in the list)

In [16]:
# Define an empty DataFrame and set the columns as date, name, amount, and category
df2 = pd.DataFrame(columns=['date','name','amount','category'])
# Retrieve all the transactions
transaction_response['transactions']
# Populate the transactions DataFrame with the transactions data
i=0
for response in transaction_response['transactions']:
     
     df2.loc[i]= [response["date"],response["name"],response["amount"],response["category"][0]]
     i=i+1
# Display sample data from the DataFrame    
df2.head()

,date,name,amount,category
0,2020-06-29,United Airlines,500,Travel
1,2020-06-27,Uber 072515 SF**POOL**,6.33,Travel
2,2020-06-24,Tectra Inc,500,Food and Drink
3,2020-06-23,AUTOMATIC PAYMENT - THANK,2078.5,Payment
4,2020-06-23,KFC,500,Food and Drink


In [17]:
### 4. Convert the data types to the appropriate types 

# (i.e. datetimeindex for the date and float for the amount)
df2.amount = df2.amount.astype(float)
df2['date'] = pd.to_datetime(df2['date'])


In [18]:
# Display data types
df2.dtypes


date        datetime64[ns]
name                object
amount             float64
category            object
dtype: object

In [19]:
# Make any required data type transformation
df2 = df2.astype({"name": str, "category": str})

In [20]:
# Set the date column as index
df2.set_index('date',inplace=True)
# Display sample data
df2.head()

,name,amount,category
date,,,
2020-06-29,United Airlines,500.00,Travel
2020-06-27,Uber 072515 SF**POOL**,6.33,Travel
2020-06-24,Tectra Inc,500.00,Food and Drink
2020-06-23,AUTOMATIC PAYMENT - THANK,2078.50,Payment
2020-06-23,KFC,500.00,Food and Drink


---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:

1. Determine the previous year's gross income and print the results

2. Determine the current monthly income and print the results

3. Determine the projected yearly income and print the results

In [23]:
# Fetch de income data
income_data =client.Income.get(access_token)
# Print the income data
# print(json.dumps(income_data["income"],indent=4, sort_keys=True))
pretty_print_response(income_data["income"])

{
    "income_streams": [
        {
            "confidence": 0.99,
            "days": 690,
            "monthly_income": 500,
            "name": "UNITED AIRLINES"
        }
    ],
    "last_year_income": 6000,
    "last_year_income_before_tax": 7285,
    "max_number_of_overlapping_income_streams": 1,
    "number_of_income_streams": 1,
    "projected_yearly_income": 6085,
    "projected_yearly_income_before_tax": 7389
}


In [26]:
# Determine the previous year's gross income and print the results
prev_yea_g_income= income_data["income"]["last_year_income_before_tax"]
print(f'previous years gross income ={prev_yea_g_income}')

previous years gross income =7285


In [39]:
# Determine the current monthly income and print the results
mon_g_income=income_data["income"]["income_streams"][0]["monthly_income"]
print(f'current monthly income = {mon_g_income}')

current monthly income = 500


In [42]:
# Determine the projected yearly income and print the results
P_yearly_income= income_data["income"]["projected_yearly_income"]
print(f'Projected yearly income = {P_yearly_income}')

Projected yearly income = 6085


---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)

2. Generate a bar chart with the number of transactions for each category

3. Calulate the expenses per month

4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# Compute the total expenses per category

# Display the expenses per catefory


In [ ]:
# Create a spending categories pie chart.


### Calculate the expenses per month

In [ ]:
# Create a DataFrame with the total expenses

# Display sample data


In [ ]:
# Create a spending per month bar chart
